# Import necessary packages

In [1]:
# Import necessary libraries
import os # For interacting with the operating system
import sys # For system-specific parameters and functions
import math # For mathematical operations
import numpy as np # For numerical operations
import pandas as pd # For data manipulation and analysis
from tqdm import tqdm # For progress bars
import matplotlib.pyplot as plt # For data visualization
import warnings # For ignoring warnings
from pathlib import PurePath    # For object-oriented file path
warnings.filterwarnings("ignore") # Ignore warning messages

# Set inputs for this phase

In [2]:
# Test threshold values

# Test 1: Circle with center at release point and radius 1 times experiment area radius
# threshold = 1

# Test 2: Circle with center at release point and radius 2 times experiment area radius
threshold = 2

# Test 3: Circle with center at release point and radius 4 times experiment area radius
# threshold = 4

# Egnore events less than 1 minute

# Test 1: Exclude Positive and Negative events with duration < 1 minute during official testing
duration = 60 # seconds


print("Using {}x experiment area radius as experimental range".format(threshold))
print("Excluding events with duration < {} seconds".format(duration))

Using 2x experiment area radius as experimental range
Excluding events with duration < 60 seconds


# 1. True Events Detection

This section is using wind speed conversion model to detect all real events, which requires three steps:
- step 1. Find all candidate events 
    Identify the following periods:
    - Non-zero release periods(0-segments): Stanford documented gas release > 0 kg gas/hr.
    - Zero release periods: Stanford documented gas release = 0 kg gas/hr.
- step 2. Find all true events
    Determine the gas clearance period. For each non-zero release period:
    - Apply a wind transport model to ascertain when gas from non-zero releases reaches a distance threshold of 2x the experimental site radius
    - Adjust the time boundaries during release periods to accommodate the gas clearance period



## 1.1 Find all candidate events

In [3]:
# Read in the latest valid true data

new_valid_true_data = pd.DataFrame(columns=["Datetime (UTC)", "True Release Rate (kg/h)", "tag"])

for date in pd.date_range(
    start=pd.to_datetime("2022-10-10"),
    end=pd.to_datetime("2022-11-30"),
    freq="D",
):
    print("Read file: ", date.date(), "...")
    filename = PurePath(
        f"../../assets/valid_true_data_pad_daily/valid_true_data_pad_daily_{date.date()}.csv"
    )
    if os.path.exists(filename):
        new_valid_true_data = pd.concat(
            [new_valid_true_data, pd.read_csv(filename)], axis=0, ignore_index=True
        )


# Create a new dataframe to store the valid true data with the required columns
valid_true_data = pd.DataFrame(columns=["Datetime (UTC)", "True Release Rate (kg/h)", "tag"])
# Convert the datetime column to datetime format and copy the true release rate column
valid_true_data["Datetime (UTC)"] = pd.to_datetime(new_valid_true_data["Datetime (UTC)"])
valid_true_data["True Release Rate (kg/h)"] = new_valid_true_data["True Release Rate (kg/h)"]
valid_true_data["tag"] = new_valid_true_data["tag"]

# the internal testing period will not influence the final results, thus, we can fill NA with 0
# valid_true_data.fillna(0, inplace=True)

Read file:  2022-10-10 ...
Read file:  2022-10-11 ...
Read file:  2022-10-12 ...
Read file:  2022-10-13 ...
Read file:  2022-10-14 ...
Read file:  2022-10-15 ...
Read file:  2022-10-16 ...
Read file:  2022-10-17 ...
Read file:  2022-10-18 ...
Read file:  2022-10-19 ...
Read file:  2022-10-20 ...
Read file:  2022-10-21 ...
Read file:  2022-10-22 ...
Read file:  2022-10-23 ...
Read file:  2022-10-24 ...
Read file:  2022-10-25 ...
Read file:  2022-10-26 ...
Read file:  2022-10-27 ...
Read file:  2022-10-28 ...
Read file:  2022-10-29 ...
Read file:  2022-10-30 ...
Read file:  2022-10-31 ...
Read file:  2022-11-01 ...
Read file:  2022-11-02 ...
Read file:  2022-11-03 ...
Read file:  2022-11-04 ...
Read file:  2022-11-05 ...
Read file:  2022-11-06 ...
Read file:  2022-11-07 ...
Read file:  2022-11-08 ...
Read file:  2022-11-09 ...
Read file:  2022-11-10 ...
Read file:  2022-11-11 ...
Read file:  2022-11-12 ...
Read file:  2022-11-13 ...
Read file:  2022-11-14 ...
Read file:  2022-11-15 ...
R

### 1.1.1 Find all candidate events(camera-based events)

Negative events(zero release periods): 
- all seconds in zero release with 0 kg gas/hr release rate
- all seconds in zero release is continuty in time;
- all seconds in zero release cannot span two dates

Positive events(non-zero release periods)

- all seconds in zero release with more than 0 kg gas/hr release rate
- all seconds in zero release is continuty in time;
- all seconds in zero release cannot span two dates

N/A events(N/A release periods)
- the release rate of all seconds in zero release is N/A
- all seconds in zero release is continuty in time;
- all seconds in zero release cannot span two dates

In [4]:
def find_events(df):
    """
    zero release periods: 
        - all seconds in zero release with 0 kg gas/hr release rate
        - all seconds in zero release is continuty in time;
        - all seconds in zero release cannot span two dates
    """

    df["date"] = df["Datetime (UTC)"].dt.date
    zero_release_peroids = pd.DataFrame(columns=["ManualStartDateTime", "ManualEndDateTime", "Label"])
    non_zero_release_peroids = pd.DataFrame(columns=["ManualStartDateTime", "ManualEndDateTime", "Label"])
    na_release_peroids = pd.DataFrame(columns=["ManualStartDateTime", "ManualEndDateTime", "Label"])

    print("Find Camera Based Events daily")
    for date in pd.date_range(start=pd.to_datetime("2022-10-10").date(), 
                              end=pd.to_datetime("2022-11-30").date(), 
                              freq="D"):
        print("current date: {}".format(date), end=" ")
        before_num_zero = zero_release_peroids.shape[0]
        before_num_non_zero = non_zero_release_peroids.shape[0]
        before_num_na = na_release_peroids.shape[0]

        daily_df = df[df["date"] == date]
        daily_df.reset_index(inplace=True, drop=True)
        i = 0
        while(i < daily_df.shape[0]):
            if daily_df.loc[i, "True Release Rate (kg/h)"] == 0:
                start = i
                while(i < daily_df.shape[0] and daily_df.loc[i, "True Release Rate (kg/h)"] == 0):
                    i += 1
                end = i - 1
                zero_release_peroids.loc[zero_release_peroids.shape[0]] = [daily_df.loc[start, "Datetime (UTC)"], 
                                                                           daily_df.loc[end, "Datetime (UTC)"], 
                                                                           "N"]
            elif daily_df.loc[i, "True Release Rate (kg/h)"] > 0:
                start = i
                while(i < daily_df.shape[0] and daily_df.loc[i, "True Release Rate (kg/h)"] > 0):
                    i += 1
                end = i - 1
                non_zero_release_peroids.loc[non_zero_release_peroids.shape[0]] = [daily_df.loc[start, "Datetime (UTC)"], 
                                                                                   daily_df.loc[end, "Datetime (UTC)"], 
                                                                                   "P"]
            elif pd.isna(daily_df.loc[i, "True Release Rate (kg/h)"]):
                start = i
                while(i < daily_df.shape[0] and pd.isna(daily_df.loc[i, "True Release Rate (kg/h)"])):
                    i += 1
                end = i - 1
                na_release_peroids.loc[na_release_peroids.shape[0]] = [daily_df.loc[start, "Datetime (UTC)"], 
                                                                       daily_df.loc[end, "Datetime (UTC)"], 
                                                                       str("NA")]


        after_num_zero = zero_release_peroids.shape[0]
        after_num_non_zero = non_zero_release_peroids.shape[0]
        after_num_na = na_release_peroids.shape[0]

        print("zero release periods: {}".format(after_num_zero - before_num_zero), end=" ")
        print("non zero release periods: {}".format(after_num_non_zero - before_num_non_zero), end=" ")
        print("na release periods: {}".format(after_num_na - before_num_na))
        
    print("Find Camera Based Events finished")
    print("zero release periods: {}".format(zero_release_peroids.shape[0]))
    print("non zero release periods: {}".format(non_zero_release_peroids.shape[0]))
    print("na release periods: {}".format(na_release_peroids.shape[0]))

    return zero_release_peroids, non_zero_release_peroids, na_release_peroids


zero_release_periods, non_zero_release_periods, na_release_periods = find_events(valid_true_data)

Find Camera Based Events daily
current date: 2022-10-10 00:00:00 zero release periods: 5 non zero release periods: 5 na release periods: 2
current date: 2022-10-11 00:00:00 zero release periods: 5 non zero release periods: 5 na release periods: 3
current date: 2022-10-12 00:00:00 zero release periods: 7 non zero release periods: 5 na release periods: 3
current date: 2022-10-13 00:00:00 zero release periods: 4 non zero release periods: 2 na release periods: 2
current date: 2022-10-14 00:00:00 zero release periods: 0 non zero release periods: 0 na release periods: 1
current date: 2022-10-15 00:00:00 zero release periods: 1 non zero release periods: 0 na release periods: 0
current date: 2022-10-16 00:00:00 zero release periods: 1 non zero release periods: 0 na release periods: 0
current date: 2022-10-17 00:00:00 zero release periods: 3 non zero release periods: 0 na release periods: 2
current date: 2022-10-18 00:00:00 zero release periods: 0 non zero release periods: 0 na release periods:

### 1.1.2 Filter camera-based events and save

Exclude Release Data:
- Dates: 2022-10-14, 2022-10-18 to 2022-10-22
- Reason: Internal testing period


Filter Stanford Events:
- Exclude Positive and Negative events with duration < 1 minute during official testing
- Reason: To eliminate brief gas puffs between releases

In [5]:
zero_dates = zero_release_periods["ManualStartDateTime"].dt.date.values
non_zero_dates = non_zero_release_periods["ManualStartDateTime"].dt.date.values
na_dates = na_release_periods["ManualStartDateTime"].dt.date.values

zero_keep_index = zero_release_periods.index.values
non_zero_keep_index = non_zero_release_periods.index.values
na_keep_index = na_release_periods.index.values

# Filter out 2022-10-14
zero_keep_index[zero_dates == pd.to_datetime("2022-10-14").date()] = -1
non_zero_keep_index[non_zero_dates == pd.to_datetime("2022-10-14").date()] = -1
na_keep_index[na_dates == pd.to_datetime("2022-10-14").date()] = -1

# Filter out dates from 2022-10-18 to 2022-10-22
zero_keep_index[(zero_dates >= pd.to_datetime("2022-10-18").date()) & (zero_dates <= pd.to_datetime("2022-10-22").date())] = -1
non_zero_keep_index[(non_zero_dates >= pd.to_datetime("2022-10-18").date()) & (non_zero_dates <= pd.to_datetime("2022-10-22").date())] = -1
na_keep_index[(na_dates >= pd.to_datetime("2022-10-18").date()) & (na_dates <= pd.to_datetime("2022-10-22").date())] = -1

# zero_keep_index = zero_keep_index[zero_keep_index != -1]
# non_zero_keep_index = non_zero_keep_index[non_zero_keep_index != -1]
# na_keep_index = na_keep_index[na_keep_index != -1]

zero_release_periods_processed = zero_release_periods.loc[zero_keep_index != 1, :]
non_zero_release_periods_processed = non_zero_release_periods.loc[non_zero_keep_index != 1, :]
na_release_periods_processed = na_release_periods.loc[na_keep_index != 1, :]

zero_release_periods_processed.reset_index(inplace=True, drop=True)
non_zero_release_periods_processed.reset_index(inplace=True, drop=True)
na_release_periods_processed.reset_index(inplace=True, drop=True)


# add 2022-10-14, 2022-10-18 to 2022-10-22 back to the na release periods
n = na_release_periods_processed.shape[0]
na_release_periods_processed.loc[n, :] = [pd.to_datetime("2022-10-14 00:00:00"), pd.to_datetime("2022-10-14 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+1, :] = [pd.to_datetime("2022-10-18 00:00:00"), pd.to_datetime("2022-10-18 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+2, :] = [pd.to_datetime("2022-10-19 00:00:00"), pd.to_datetime("2022-10-19 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+3, :] = [pd.to_datetime("2022-10-20 00:00:00"), pd.to_datetime("2022-10-20 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+4, :] = [pd.to_datetime("2022-10-21 00:00:00"), pd.to_datetime("2022-10-21 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+5, :] = [pd.to_datetime("2022-10-22 00:00:00"), pd.to_datetime("2022-10-22 23:59:59"), str("NA")]


zero_release_periods.sort_values(by="ManualStartDateTime", inplace=True)
non_zero_release_periods.sort_values(by="ManualStartDateTime", inplace=True)
na_release_periods.sort_values(by="ManualStartDateTime", inplace=True)

zero_release_periods_processed.reset_index(inplace=True, drop=True)
non_zero_release_periods_processed.reset_index(inplace=True, drop=True)
na_release_periods_processed.reset_index(inplace=True, drop=True)


zero_release_periods_processed_camera = zero_release_periods_processed.copy()
non_zero_release_periods_processed_camera = non_zero_release_periods_processed.copy()
na_release_periods_processed_camera = na_release_periods_processed.copy()


# filter out the release periods with duration less than 1 minute
for i in range(zero_release_periods_processed_camera.shape[0]):
    if (zero_release_periods_processed_camera.loc[i, "ManualEndDateTime"] - zero_release_periods_processed_camera.loc[i, "ManualStartDateTime"]).seconds < duration:
        zero_release_periods_processed_camera.loc[i, "Label"] = str("NA")
for i in range(non_zero_release_periods_processed_camera.shape[0]):
    if (non_zero_release_periods_processed_camera.loc[i, "ManualEndDateTime"] - non_zero_release_periods_processed_camera.loc[i, "ManualStartDateTime"]).seconds < duration:
        non_zero_release_periods_processed_camera.loc[i, "Label"] = str("NA")

# combine all the release periods
all_release_periods = pd.concat([zero_release_periods_processed_camera, non_zero_release_periods_processed_camera, na_release_periods_processed_camera], axis=0)
all_release_periods.sort_values(by="ManualStartDateTime", inplace=True)
all_release_periods.reset_index(inplace=True, drop=True)


zero_release_periods_processed_camera = all_release_periods[all_release_periods["Label"] == "N"].reset_index(drop=True)
non_zero_release_periods_processed_camera = all_release_periods[all_release_periods["Label"] == "P"].reset_index(drop=True)
na_release_periods_processed_camera = all_release_periods[all_release_periods["Label"] == str("NA")].reset_index(drop=True)


# save the zero, non-zero, na release periods
zero_release_periods_processed_camera.to_csv(
    PurePath("../../assets/events_PN/candidate_event_duration={}xseconds_N.csv".format(duration)), index=False)
non_zero_release_periods_processed_camera.to_csv(
    PurePath("../../assets/events_PN/candidate_event_duration={}xseconds_P.csv".format(duration)), index=False)
na_release_periods_processed_camera.to_csv(
    PurePath("../../assets/events_PN/candidate_event_duration={}xseconds_NA.csv".format(duration)), index=False)


In [6]:
na_release_periods_processed_camera.tail(10)

,ManualStartDateTime,ManualEndDateTime,Label
372,2022-11-21 22:22:07,2022-11-21 22:37:40,NA
373,2022-11-22 16:36:00,2022-11-22 16:40:00,NA
374,2022-11-22 16:40:01,2022-11-22 16:40:50,NA
375,2022-11-22 20:46:15,2022-11-22 20:46:19,NA
376,2022-11-22 22:17:58,2022-11-22 22:18:07,NA
377,2022-11-22 22:18:08,2022-11-22 22:18:38,NA
378,2022-11-23 16:18:43,2022-11-23 16:28:00,NA
379,2022-11-23 22:02:04,2022-11-23 22:10:21,NA
380,2022-11-28 16:24:19,2022-11-28 16:30:00,NA
381,2022-11-29 17:25:23,2022-11-29 17:29:21,NA


## 1.2 Find all True Events

The prescribed method for this stage entails the following computational procedure:

1. Determine the experiment_area_radius, representing the maximum extent from the release point to the experimental boundaries.
2. Retrieve wind velocity data (u_east and u_north components) from October 10, 2022, to November 30, 2022.
3. Determine the gas clearance period. For each non-zero release period: 
    - Apply a wind transport model to ascertain when gas from non-zero releases reaches a distance threshold of 2x the experimental site radius, 
    - Adjust the time boundaries during release periods to accommodate the gas clearance period

### 1.2.1 Calculate Experimental Range

In [7]:
# Get the data of the release point and the experiment area
# Release point location, sensor location, function to get distance based on latitude and longitude
release_location = [
    (32.8218205, -111.7857730)
]

experiment_area_location = [
    (32.8221200, -111.7862250),
    (32.8212826, -111.7861378),
    (32.8212806, -111.7851762),
    (32.8220614, -111.7849958)
]

# Function to calculate distance based on latitude and longitude
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140  # Equatorial radius
    rb = 6356755  # Polar radius
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # Convert angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
 
    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance, 4)
    return distance

# Function to calculate angle
# Angle deviation from true north
def getDegree(latA, lonA, latB, lonB):
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
    dLon = radLonB - radLonA
    y = math.sin(dLon) * math.cos(radLatB)
    x = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = math.degrees(math.atan2(y, x))
    brng = round((brng + 360) % 360, 4)
    brng = int(brng)

    return brng

# Release point latitude and longitude information
release_latlng = release_location[0]

# Calculate the maximum distance (radius) from the experiment area to the release point
experiment_area_radius = max([getDistance(release_latlng[0], 
                                  release_latlng[1], 
                                  experiment_area_location[i][0],
                                  experiment_area_location[i][1]) for i in range(len(experiment_area_location))])

print("Release point (Stack + Flowskid) location: ")
print("\tLatLon  : {}, {}".format(release_latlng[0], release_latlng[1]))
print("Experiment Area Location:")
print("\tLatLon : ({}, {})->({}, {})".format(experiment_area_location[0][0], experiment_area_location[0][1], experiment_area_location[1][0], experiment_area_location[1][1]))
distance_threshold = experiment_area_radius * threshold
print("Distance threshold({}xradius): {}".format(threshold, distance_threshold))

Release point (Stack + Flowskid) location: 
	LatLon  : 32.8218205, -111.785773
Experiment Area Location:
	LatLon : (32.82212, -111.786225)->(32.8212826, -111.7861378)
Distance threshold(2xradius): 163.8072


### 1.2.2 Compute Accumulated Wind Components

In [8]:
# Get wind speed data, wind speed data stored in ../../assets/all_wind_data, contains 5 files
# Read in wind speed data from a csv files
# 1. set root path and file name list 

root_path = PurePath("../../assets/all_wind_data/")
file_name_list = [
    "all_wind_1.csv",
    "all_wind_2.csv",
    "all_wind_3.csv",
    "all_wind_4.csv",
    "all_wind_5.csv"
]

# initialize a DataFrame to store all wind speed data, columns = [datetime, windspeed, winddirection, day, month, u_east, u_north]
wind_speed = pd.DataFrame(columns=["datetime", "windspeed", "winddirection", "day", "month", "u_east", "u_north"])
# 2. read in all wind speed data
for file_name in file_name_list:
    tmp_df = pd.read_csv(PurePath(root_path, file_name), low_memory=False)
    tmp_df["datetime"] = pd.to_datetime(tmp_df["datetime"])
    # keep only the columns we need
    tmp_df = tmp_df[["datetime", "windspeed", "winddirection", "day", "month", "u_east", "u_north"]]
    # append to all_wind_speed
    wind_speed = pd.concat([wind_speed, tmp_df], ignore_index=True)

# 3. sort by datetime
wind_speed.sort_values(by="datetime", inplace=True)
wind_speed = wind_speed.reset_index(drop=True)
# Calculate the wind speed magnitude (u_norm) using the east and north components (u_east and u_north)
wind_speed["u_norm"] = np.sqrt(wind_speed["u_east"] ** 2 + wind_speed["u_north"] ** 2)
wind_speed["u_east_component"] = wind_speed["u_east"].cumsum()
wind_speed["u_north_component"] = wind_speed["u_north"].cumsum()

# Fill in any missing values using the previous value (pad method)
wind_speed.fillna(method="pad", inplace=True)

### 1.2.3 Determine the gas clearance period

In [9]:

def determine_true_endtime_based_on_wind_transpose_model(wind_speed, starttime, endtime, targettime, distance_threshold, is_last_event=False):
    """
    Determine the true R0 based on the wind transpose model
    Args:
        wind_speed: wind speed data
        starttime: start time of the event
        endtime: end time of the event
        targettime: start time of next event
        distance_threshold: obersevation distance threshold
        is_last_event: whether this is the last event
    Returns:
        true end time of the event
    """

    # Calculate the travel distance of wind particles from starttime to targettime for each second
    internal_df_windspeed = wind_speed.loc[wind_speed['datetime'].between(starttime, targettime, inclusive="both"), :]
    internal_df_windspeed.reset_index(drop=True, inplace=True)
    internal_df_windspeed["u_east_component"] = internal_df_windspeed["u_east_component"] - internal_df_windspeed.loc[0, "u_east_component"]
    internal_df_windspeed["u_north_component"] = internal_df_windspeed["u_north_component"] - internal_df_windspeed.loc[0, "u_north_component"]

    # Calculate the wind transposition distance (u_east ** 2 + u_north ** 2) from starttime to endtime for all instances until targettime.
    calculate_u = internal_df_windspeed.loc[internal_df_windspeed['datetime'].between(starttime, endtime), ["u_east_component", "u_north_component"]].values
    end_u = internal_df_windspeed.loc[internal_df_windspeed['datetime'] <= targettime, ["u_east_component", "u_north_component"]].values
    
    if end_u.shape[0] > 0:
        end_u = end_u[-1, :]
    endpoints = end_u - calculate_u
    endpoints = np.power(np.power(endpoints, 2).sum(axis=1), 0.5)
    if len(endpoints) == 0:
        endpoints = np.array([0])

    
    # if current candidate event is the last event of a day and the endpoint is less than distance_threshold, return targettime
    if endpoints.min() < distance_threshold and is_last_event:
        return targettime
    # if current candidate event is not the last event of a day and the endpoint is less than threshold, return None, that's is a False R0
    elif endpoints.min() < distance_threshold and is_last_event == False:
        return None
    # if the endpoint is greater than distance_threshold, we need to find the exact R0, which is the first time that endpoint is greater than distance_threshold
    else:
        end_u = internal_df_windspeed.loc[internal_df_windspeed['datetime'].between(endtime, targettime), ["u_east_component", "u_north_component"]].values
        m1, n1 = calculate_u.shape
        m2, n2 = end_u.shape
        calculate_u = calculate_u.reshape((m1, 1, n1))
        end_u = end_u.reshape((1, m2, n2))
        calculate_u = np.repeat(calculate_u, m2, axis=1)
        end_u = np.repeat(end_u, m1, axis=0)

        endpoints = end_u - calculate_u
        endpoints = np.power(np.power(endpoints, 2).sum(axis=2), 0.5)
        endpoints = endpoints.min(axis=0)

        # find the first time that endpoint is greater than distance_threshold
        index = np.where(endpoints > distance_threshold)[0][0]

        return endtime + pd.Timedelta(seconds=index)
    
    
def get_candidate_events_by_determined_true_r0(wind_speed, non_zero_release_periods):
    
    gas_clearance_periods = non_zero_release_periods.copy()
    event_no = 0
    gas_clearance_periods["event_no"] = 0
    n = gas_clearance_periods.shape[0]
    print("Total number of candidate events: {}".format(n))
    merged_number = 0
    for i in range(1, n):
        curr_candicate_event = gas_clearance_periods.loc[i - 1, ["ManualStartDateTime", "ManualEndDateTime", "event_no"]]
        next_candicate_event = gas_clearance_periods.loc[i, ["ManualStartDateTime", "ManualEndDateTime", "event_no"]]

        curr_date = pd.to_datetime(curr_candicate_event.ManualEndDateTime).date()
        next_date = pd.to_datetime(next_candicate_event.ManualStartDateTime).date()
        
        
        if curr_date != next_date:
            gas_clearance_periods.loc[i - 1, "event_no"] = event_no
            event_no = event_no + 1
            true_r0 = determine_true_endtime_based_on_wind_transpose_model(wind_speed, 
                                                                      pd.to_datetime(curr_candicate_event.ManualStartDateTime), 
                                                                      pd.to_datetime(curr_candicate_event.ManualEndDateTime), 
                                                                      pd.to_datetime(curr_date) + pd.Timedelta("1d") - pd.Timedelta("1s"), 
                                                                      distance_threshold, is_last_event=True)
            
            gas_clearance_periods.loc[i-1, "ManualEndDateTime"] = true_r0
        else:
            starttime = pd.to_datetime(curr_candicate_event.ManualStartDateTime)
            endtime = pd.to_datetime(curr_candicate_event.ManualEndDateTime) 
            targettime = pd.to_datetime(next_candicate_event.ManualStartDateTime)
            true_r0 = determine_true_endtime_based_on_wind_transpose_model(wind_speed, starttime, endtime, targettime, distance_threshold)
            if true_r0:
                gas_clearance_periods.loc[i - 1, "event_no"] = event_no
                event_no = event_no + 1
                gas_clearance_periods.loc[i-1, "ManualEndDateTime"] = true_r0
            else:
                merged_number = merged_number + 1
                gas_clearance_periods.loc[i - 1, "event_no"] = event_no
    
    print("The number of merged events is {}".format(merged_number))
    # The last candidate event
    gas_clearance_periods.loc[n - 1, "event_no"] = event_no
    event_no = event_no + 1
    print("After wind transpose model with distance_threshold = {}xradius, the number of events is {}".format(threshold, event_no))
    return gas_clearance_periods
 

gas_clearance_periods = get_candidate_events_by_determined_true_r0(wind_speed, non_zero_release_periods_processed)

Total number of candidate events: 328
The number of merged events is 220
After wind transpose model with distance_threshold = 2xradius, the number of events is 108


### 1.2.4 Adjust time boundaries for non-zero release periods
Non-zero releases with time boundaries adjusted for the gas clearance period

In [10]:
# Merge events
def merge_events(gas_clearance_periods):
    """
    Merge events with the same event number, and update the ManualStartDateTime, ManualEndDateTime 
    and True Release Rate (kg/h) of the event.
    Args:
        non_zero_release_periods: pd.DataFrame, a set of events to merge
    Returns:
        pd.DataFrame: a set of events have been merged
    """

    # Merge events

    # Merge event numbers
    # Combine events with the same `event_no` into one event, 
    # reset the left 0 time of the event, the right 0 time of the event, and the gas average release rate of the event
    merge_envent = gas_clearance_periods.drop_duplicates(subset=['event_no'])
    merge_envent.reset_index(drop=True, inplace=True)

    for no in gas_clearance_periods["event_no"].unique():
        enven_df = gas_clearance_periods.loc[gas_clearance_periods.event_no == no, :]
        ManualStartDateTime = enven_df.ManualStartDateTime.min()
        ManualEndDateTime = enven_df.ManualEndDateTime.max()
        
        Rate = valid_true_data.loc[
            (valid_true_data["Datetime (UTC)"] >= ManualStartDateTime) & \
                (valid_true_data["Datetime (UTC)"] <= ManualEndDateTime),
            'True Release Rate (kg/h)'].mean()
        merge_envent.loc[merge_envent.event_no == no, 'ManualStartDateTime'] = ManualStartDateTime
        merge_envent.loc[merge_envent.event_no == no, 'ManualEndDateTime'] = ManualEndDateTime


    gas_clearance_periods_a = merge_envent[['ManualStartDateTime', 'ManualEndDateTime', 'Label']]
    gas_clearance_periods_a.reset_index(drop=True, inplace=True)

    return gas_clearance_periods_a

gas_clearance_periods_merged = merge_events(gas_clearance_periods)

### 1.2.5 Define Stanford-Defined Events:

- Positive Events: Non-zero releases with time boundaries adjusted for the gas clearance period
- Negative Events: Zero releases with time boundaries adjusted for the gas clearance period

In [11]:
# 1. make "True Realse Rate (kg/h)" of the seconds fall in adjusted non-zero release periods to 999.0

for i in range(gas_clearance_periods_merged.shape[0]):
    starttime = gas_clearance_periods_merged.loc[i, "ManualStartDateTime"]
    endtime = gas_clearance_periods_merged.loc[i, "ManualEndDateTime"]
    valid_true_data.loc[(valid_true_data["Datetime (UTC)"] >= starttime) & (valid_true_data["Datetime (UTC)"] <= endtime), "True Release Rate (kg/h)"] = 999.0


# find all sensor-based events
zero_release_periods, non_zero_release_periods, na_release_periods = find_events(valid_true_data)

Find Camera Based Events daily
current date: 2022-10-10 00:00:00 zero release periods: 2 non zero release periods: 2 na release periods: 2
current date: 2022-10-11 00:00:00 zero release periods: 3 non zero release periods: 3 na release periods: 3
current date: 2022-10-12 00:00:00 zero release periods: 4 non zero release periods: 2 na release periods: 3
current date: 2022-10-13 00:00:00 zero release periods: 3 non zero release periods: 1 na release periods: 2
current date: 2022-10-14 00:00:00 zero release periods: 0 non zero release periods: 0 na release periods: 1
current date: 2022-10-15 00:00:00 zero release periods: 1 non zero release periods: 0 na release periods: 0
current date: 2022-10-16 00:00:00 zero release periods: 1 non zero release periods: 0 na release periods: 0
current date: 2022-10-17 00:00:00 zero release periods: 3 non zero release periods: 0 na release periods: 2
current date: 2022-10-18 00:00:00 zero release periods: 0 non zero release periods: 0 na release periods:

### 1.2.6 Filter sensor-based events and save

Exclude Release Data:
- Dates: 2022-10-14, 2022-10-18 to 2022-10-22
- Reason: Internal testing period


Filter Stanford Events:
- Exclude Positive and Negative events with duration < 1 minute during official testing
- Reason: To eliminate brief gas puffs between releases 


In [12]:
zero_dates = zero_release_periods["ManualStartDateTime"].dt.date.values
non_zero_dates = non_zero_release_periods["ManualStartDateTime"].dt.date.values
na_dates = na_release_periods["ManualStartDateTime"].dt.date.values

zero_keep_index = zero_release_periods.index.values
non_zero_keep_index = non_zero_release_periods.index.values
na_keep_index = na_release_periods.index.values

# Filter out 2022-10-14
zero_keep_index[zero_dates == pd.to_datetime("2022-10-14").date()] = -1
non_zero_keep_index[non_zero_dates == pd.to_datetime("2022-10-14").date()] = -1
na_keep_index[na_dates == pd.to_datetime("2022-10-14").date()] = -1

# Filter out dates from 2022-10-18 to 2022-10-22
zero_keep_index[(zero_dates >= pd.to_datetime("2022-10-18").date()) & (zero_dates <= pd.to_datetime("2022-10-22").date())] = -1
non_zero_keep_index[(non_zero_dates >= pd.to_datetime("2022-10-18").date()) & (non_zero_dates <= pd.to_datetime("2022-10-22").date())] = -1
na_keep_index[(na_dates >= pd.to_datetime("2022-10-18").date()) & (na_dates <= pd.to_datetime("2022-10-22").date())] = -1

# zero_keep_index = zero_keep_index[zero_keep_index != -1]
# non_zero_keep_index = non_zero_keep_index[non_zero_keep_index != -1]
# na_keep_index = na_keep_index[na_keep_index != -1]

zero_release_periods_processed = zero_release_periods.loc[zero_keep_index != -1, :]
non_zero_release_periods_processed = non_zero_release_periods.loc[non_zero_keep_index != -1, :]
na_release_periods_processed = na_release_periods.loc[na_keep_index != -1, :]

zero_release_periods_processed.reset_index(inplace=True, drop=True)
non_zero_release_periods_processed.reset_index(inplace=True, drop=True)
na_release_periods_processed.reset_index(inplace=True, drop=True)


# add 2022-10-14, 2022-10-18 to 2022-10-22 back to the na release periods
n = na_release_periods_processed.shape[0]
na_release_periods_processed.loc[n, :] = [pd.to_datetime("2022-10-14 00:00:00"), pd.to_datetime("2022-10-14 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+1, :] = [pd.to_datetime("2022-10-18 00:00:00"), pd.to_datetime("2022-10-18 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+2, :] = [pd.to_datetime("2022-10-19 00:00:00"), pd.to_datetime("2022-10-19 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+3, :] = [pd.to_datetime("2022-10-20 00:00:00"), pd.to_datetime("2022-10-20 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+4, :] = [pd.to_datetime("2022-10-21 00:00:00"), pd.to_datetime("2022-10-21 23:59:59"), str("NA")]
na_release_periods_processed.loc[n+5, :] = [pd.to_datetime("2022-10-22 00:00:00"), pd.to_datetime("2022-10-22 23:59:59"), str("NA")]


zero_release_periods.sort_values(by="ManualStartDateTime", inplace=True)
non_zero_release_periods.sort_values(by="ManualStartDateTime", inplace=True)
na_release_periods.sort_values(by="ManualStartDateTime", inplace=True)

zero_release_periods_processed.reset_index(inplace=True, drop=True)
non_zero_release_periods_processed.reset_index(inplace=True, drop=True)
na_release_periods_processed.reset_index(inplace=True, drop=True)


zero_release_periods_processed_sensor = zero_release_periods_processed.copy()
non_zero_release_periods_processed_sensor = non_zero_release_periods_processed.copy()
na_release_periods_processed_sensor = na_release_periods_processed.copy()


# filter out the release periods with duration less than 1 minute
for i in range(zero_release_periods_processed_sensor.shape[0]):
    if (zero_release_periods_processed_sensor.loc[i, "ManualEndDateTime"] - zero_release_periods_processed_sensor.loc[i, "ManualStartDateTime"]).seconds < duration:
        zero_release_periods_processed_sensor.loc[i, "Label"] = str("NA")
for i in range(non_zero_release_periods_processed_sensor.shape[0]):
    if (non_zero_release_periods_processed_sensor.loc[i, "ManualEndDateTime"] - non_zero_release_periods_processed_sensor.loc[i, "ManualStartDateTime"]).seconds < duration:
        non_zero_release_periods_processed_sensor.loc[i, "Label"] = str("NA")

# combine all the release periods
all_release_periods = pd.concat([zero_release_periods_processed_sensor, non_zero_release_periods_processed_sensor, na_release_periods_processed_sensor], axis=0)
all_release_periods.sort_values(by="ManualStartDateTime", inplace=True)
all_release_periods.reset_index(inplace=True, drop=True)


zero_release_periods_processed_sensor = all_release_periods[all_release_periods["Label"] == "N"].reset_index(drop=True)
non_zero_release_periods_processed_sensor = all_release_periods[all_release_periods["Label"] == "P"].reset_index(drop=True)
na_release_periods_processed_sensor = all_release_periods[all_release_periods["Label"] == str("NA")].reset_index(drop=True)


# save the zero, non-zero, na release periods
zero_release_periods_processed_sensor.to_csv(
    PurePath("../../assets/events_PN/true_event_threshold={}xradius_duration={}xseconds_N.csv".format(threshold, duration)), index=False)
non_zero_release_periods_processed_sensor.to_csv(
    PurePath("../../assets/events_PN/true_event_threshold={}xradius_duration={}xseconds_P.csv".format(threshold, duration)), index=False)
na_release_periods_processed_sensor.to_csv(
    PurePath("../../assets/events_PN/true_event_threshold={}xradius_duration={}xseconds_NA.csv".format(threshold, duration)), index=False)
